In [ ]:
#install the deeppavlov library by running
!pip install deeppavlov

In [1]:
import copy
from pprint import pprint
import deeppavlov
from deeppavlov.dataset_readers.dstc2_reader import DSTC2DatasetReader
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator
from deeppavlov import build_model, deep_download

!export CUDA_VISIBLE_DEVICES=""
!echo "cuda visible devices = '"$CUDA_VISIBLE_DEVICES"'"

cuda visible devices = ''


In [2]:
data = DSTC2DatasetReader().read(data_path="tmp/my_download_of_dstc2")

2019-01-22 06:59:06.203 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from tmp/my_download_of_dstc2/dstc2-trn.jsonlist]
2019-01-22 06:59:06.470 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from tmp/my_download_of_dstc2/dstc2-val.jsonlist]
2019-01-22 06:59:06.632 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from tmp/my_download_of_dstc2/dstc2-tst.jsonlist]


In [3]:
batches_generator = DialogDatasetIterator(data, seed=1443, shuffle=True)\
                                         .gen_batches(batch_size=4, data_type='train')

In [4]:
batch = batches_generator.__next__()

In [5]:
x_batch, y_batch = batch

In [6]:
dialog_id = 0
dialog = [(x, y) for x, y in zip(x_batch[dialog_id], y_batch[dialog_id])]

turn_id = 0
print("----{}th turn----".format(turn_id)) 
pprint(dialog[turn_id], indent=8)

----0th turn----
(       {'intents': [], 'prev_resp_act': None, 'text': ''},
        {       'act': 'welcomemsg',
                'text': 'Hello, welcome to the Cambridge restaurant system. '
                        'You can ask for restaurants by area, price range or '
                        'food type. How may I help you?'})


In [7]:
for turn in dialog:
    x, y = turn
    print('::', x['text']) 
    print('>>', y['text'], '\n')

:: 
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you? 

:: traditional
>> api_call area="dontcare" food="traditional" pricerange="dontcare" 

:: traditional
>> I am sorry but there is no traditional restaurant that matches your request. 

:: italian food
>> What part of town do you have in mind? 

:: south
>> api_call area="south" food="italian" pricerange="dontcare" 

:: south
>> Pizza hut cherry hinton is a nice place in the south of town serving tasty italian food. 

:: phone number
>> The phone number of pizza hut cherry hinton is 01223 323737. 

:: post code
>> The post code of pizza hut cherry hinton is C.B 1, 7 D.Y. 

:: thank you good bye
>> You are welcome! 



In [8]:
vocab_config = {}

In [9]:
dstc2_reader_comp_config = {
    'class_name': 'dstc2_reader',
    'data_path': 'dstc2'
}
vocab_config['dataset_reader'] = dstc2_reader_comp_config

In [10]:
dialog_iterator_comp_config = {
    'class_name': 'dialog_iterator'
}
vocab_config['dataset_iterator'] = dialog_iterator_comp_config

In [11]:
dstc2_download_config = {
    'url': 'http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz',
    'subdir': 'dstc2'
}
vocab_config['metadata'] = {}
vocab_config['metadata']['download'] = [
    dstc2_download_config
]

In [12]:
vocab_config['train'] = {
    'validate_best': False,
    'test_best': False
}
vocab_config['chainer'] = {}
vocab_config['chainer']['in'] = ['utterance']
vocab_config['chainer']['in_y'] = []
vocab_config['chainer']['out'] = []

In [13]:
vocab_comp_config = {
    'class_name': 'default_vocab',
    'save_path': 'vocabs/token.dict',
    'load_path': 'vocabs/token.dict',
    'fit_on': ['utterance'],
    'level': 'token',
    'tokenizer': {
        'class_name': 'split_tokenizer'
    },

}
vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]

In [14]:
deep_download(vocab_config)

2019-01-22 06:59:08.35 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2019-01-22 06:59:08.587 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz.md5 HTTP/1.1" 200 260
2019-01-22 06:59:08.611 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz download because of matching hashes


In [15]:
dstc2_path = './dstc2'

In [16]:
from deeppavlov import train_model

train_model(vocab_config)

2019-01-22 06:59:08.627 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-trn.jsonlist]
2019-01-22 06:59:08.935 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-val.jsonlist]
2019-01-22 06:59:09.188 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-tst.jsonlist]
[nltk_data] Downloading package punkt to /Users/maxtran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maxtran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/maxtran/nltk_data...
[nltk_data]   Package perluniprops 

In [17]:
vocabs_path = './vocabs'

In [18]:
vocab_comp_config['in'] = ['utterance']
vocab_comp_config['out'] = ['utterance_token_indices']

vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]
vocab_config['chainer']['out'] = ['utterance_token_indices']

In [19]:
model = build_model(vocab_config)

2019-01-22 06:59:11.875 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/vocabs/token.dict]


In [20]:
simple_config = {}

simple_config['dataset_reader'] = dstc2_reader_comp_config
simple_config['dataset_iterator'] = dialog_iterator_comp_config
simple_config['metadata'] = {}
simple_config['metadata']['download'] = [
    dstc2_download_config
]
simple_config['chainer'] = {}
simple_config['chainer']['in'] = ['x']
simple_config['chainer']['in_y'] =['y']
simple_config['chainer']['out'] = ['y_predicted']
vocab_comp_config = {
    'class_name': 'default_vocab',
    'id': 'token_vocab',
    'load_path': 'vocabs/token.dict',
    'save_path': 'vocabs/token.dict',
    'fit_on': ['x'],
    'level': 'token',
    'tokenizer': {
        'class_name': 'split_tokenizer'
    }
}
simple_config['chainer']['pipe'] = []
simple_config['chainer']['pipe'].append(vocab_comp_config)

In [21]:
bot_comp_config = {
    'class_name': 'go_bot',
    'in': ['x'],
    'in_y': ['y'],
    'out': ['y_predicted'],
    'load_path': 'gobot_dstc2_simple/model',
    'save_path': 'gobot_dstc2_simple/model',
    'word_vocab': None,
    'bow_embedder': None,
    'embedder': None,
    'slot_filler': None,
    'template_path': 'dstc2/dstc2-templates.txt',
    'template_type': 'DualTemplate',
    'database': None,
    'api_call_action': 'api_call',
    'network_parameters': {
      'dense_size': 64,
      'hidden_size': 128,
      'learning_rate': 0.002,
      'attention_mechanism': None
    },
    'tokenizer': {
        'class_name': 'stream_spacy_tokenizer',
        'lowercase': False
    },
    'tracker': {
        'class_name': 'featurized_tracker',
        'slot_names': ['pricerange', 'this', 'area', 'food', 'name']
    },
    'main': True,
    'debug': False
}
bot_comp_config['word_vocab'] = '#token_vocab'
bot_comp_config['bow_embedder'] = {
    "class_name": "bow",
    "depth": "#token_vocab.__len__()", # here we referenced __len__() method of token_vocab component
    "with_counts": True
}

In [22]:
from deeppavlov import configs


In [23]:
slot_filler_comp_config = {
    'config_path': configs.ner.slotfill_dstc2
}
bot_comp_config['slot_filler'] = slot_filler_comp_config

simple_config['chainer']['pipe'].append(bot_comp_config)
simple_bot_train_config = {
    'batch_size': 4,
    'epochs': 10,
    'log_every_n_batches': -1,
    'log_every_n_epochs': 1,
    'metrics': ['per_item_dialog_accuracy'],
    'val_every_n_epochs': 1,
    'validation_patience': 20,
    'validate_best': False,
    'test_best': False
}
simple_config['train'] = simple_bot_train_config

In [24]:
deep_download(slot_filler_comp_config['config_path'])

2019-01-22 06:59:25.779 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2019-01-22 06:59:26.302 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/slotfill_dstc2.tar.gz.md5 HTTP/1.1" 200 442
2019-01-22 06:59:26.313 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz download because of matching hashes


In [25]:

train_model(simple_config)

2019-01-22 06:59:30.967 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-trn.jsonlist]
2019-01-22 06:59:31.342 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-val.jsonlist]
2019-01-22 06:59:31.467 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-tst.jsonlist]
2019-01-22 06:59:31.835 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/vocabs/token.dict]
2019-01-22 06:59:31.862 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/vocabs/token.dict]
2019-01-22 06:59:38.697 INFO in 'dee

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0213}, "time_spent": "0:01:13", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 20}}
{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.406}, "time_spent": "0:04:34", "loss": 1.092983921574167, "learning_rate": 0.002, "momentum": 0.9}}


2019-01-22 07:05:23.178 INFO in 'deeppavlov.core.commands.train'['train'] at line 608: New best per_item_dialog_accuracy of 0.4285
2019-01-22 07:05:23.179 INFO in 'deeppavlov.core.commands.train'['train'] at line 610: Saving model
2019-01-22 07:05:23.180 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 71: [saving model to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_simple/model]
2019-01-22 07:05:23.319 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 708: [saving parameters to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_simple/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4285}, "time_spent": "0:05:42", "epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "impatience": 0, "patience_limit": 20}}
{"train": {"epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "metrics": {"per_item_dialog_accuracy": 0.5435}, "time_spent": "0:09:05", "loss": 0.5504564134296307, "learning_rate": 0.002, "momentum": 0.9}}


2019-01-22 07:09:53.629 INFO in 'deeppavlov.core.commands.train'['train'] at line 608: New best per_item_dialog_accuracy of 0.4426
2019-01-22 07:09:53.630 INFO in 'deeppavlov.core.commands.train'['train'] at line 610: Saving model
2019-01-22 07:09:53.631 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 71: [saving model to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_simple/model]
2019-01-22 07:09:53.768 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 708: [saving parameters to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_simple/model.json]


...

2019-01-22 07:42:49.191 INFO in 'deeppavlov.core.commands.train'['train'] at line 615: Did not improve on the per_item_dialog_accuracy of 0.4649


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.454}, "time_spent": "0:43:08", "epochs_done": 9, "batches_seen": 2178, "train_examples_seen": 8703, "impatience": 5, "patience_limit": 20}}
{"train": {"epochs_done": 10, "batches_seen": 2420, "train_examples_seen": 9670, "metrics": {"per_item_dialog_accuracy": 0.6201}, "time_spent": "0:46:36", "loss": 0.25319721702147613, "learning_rate": 0.002, "momentum": 0.9}}


2019-01-22 07:47:27.706 INFO in 'deeppavlov.core.commands.train'['train'] at line 608: New best per_item_dialog_accuracy of 0.4715
2019-01-22 07:47:27.707 INFO in 'deeppavlov.core.commands.train'['train'] at line 610: Saving model
2019-01-22 07:47:27.709 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 71: [saving model to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_simple/model]
2019-01-22 07:47:27.866 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 708: [saving parameters to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_simple/model.json]
2019-01-22 07:47:27.871 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/vocabs/token.dict]
2019-01-22 07:47:27.877 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/word.dict]
2019-01-22 07:47:27.881 INF

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4715}, "time_spent": "0:47:47", "epochs_done": 10, "batches_seen": 2420, "train_examples_seen": 9670, "impatience": 0, "patience_limit": 20}}


2019-01-22 07:47:28.987 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 47: [loading model from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/model]
2019-01-22 07:47:29.10 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/model
2019-01-22 07:47:30.81 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 160: [loading templates from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-templates.txt]
2019-01-22 07:47:30.84 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 163: 46 templates loaded
2019-01-22 07:47:30.84 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 202: Calculated input size for `GoalOrientedBotNetwork` is 511
2019-01-22 07:47:30.844 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 240: [initializing `GoalOrientedBot` from saved]
2019-01-22 07:47:30.845 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 714: [load

In [26]:
model = build_model(simple_config)

2019-01-22 09:22:11.647 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/vocabs/token.dict]
2019-01-22 09:22:11.659 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/word.dict]
2019-01-22 09:22:11.665 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/tag.dict]
2019-01-22 09:22:12.891 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 47: [loading model from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/model]
2019-01-22 09:22:12.924 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/model
2019-01-22 09:22:13.490 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 160: [loading templates from /Users/maxtran/Desktop/C

In [34]:
model.reset() 

In [33]:
db_config = copy.deepcopy(simple_config)

db_config['chainer']['pipe'] = []
db_comp_config = {
    'class_name': 'sqlite_database',
    'id': 'restaurant_database', 
    'save_path': 'dstc2/resto.sqlite',
    'primary_keys': ['name'],
    'table_name': 'mytable'
}
db_config['chainer']['pipe'].append(vocab_comp_config)
db_config['chainer']['pipe'].append(db_comp_config)
bot_with_db_comp_config = copy.deepcopy(bot_comp_config)
bot_with_db_comp_config['load_path'] = 'gobot_dstc2_db/model'
bot_with_db_comp_config['save_path'] = 'gobot_dstc2_db/model'
bot_with_db_comp_config['database'] = '#restaurant_database'
db_config['chainer']['pipe'].append(bot_with_db_comp_config)

In [35]:
model = train_model(db_config)

2019-01-22 09:25:20.325 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-trn.jsonlist]
2019-01-22 09:25:20.529 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-val.jsonlist]
2019-01-22 09:25:20.657 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-tst.jsonlist]
2019-01-22 09:25:21.136 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/vocabs/token.dict]
2019-01-22 09:25:21.161 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/vocabs/token.dict]
2019-01-22 09:25:21.171 WARNING in '

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0034}, "time_spent": "0:01:10", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 20}}
{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.4009}, "time_spent": "0:04:36", "loss": 1.1325077372641603, "learning_rate": 0.002, "momentum": 0.9}}


2019-01-22 09:31:08.657 INFO in 'deeppavlov.core.commands.train'['train'] at line 608: New best per_item_dialog_accuracy of 0.4112
2019-01-22 09:31:08.658 INFO in 'deeppavlov.core.commands.train'['train'] at line 610: Saving model
2019-01-22 09:31:08.661 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 71: [saving model to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_db/model]
2019-01-22 09:31:08.810 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 708: [saving parameters to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_db/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4112}, "time_spent": "0:05:45", "epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "impatience": 0, "patience_limit": 20}}
{"train": {"epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "metrics": {"per_item_dialog_accuracy": 0.5383}, "time_spent": "0:09:19", "loss": 0.5451136353956766, "learning_rate": 0.002, "momentum": 0.9}}


2019-01-22 09:35:52.621 INFO in 'deeppavlov.core.commands.train'['train'] at line 608: New best per_item_dialog_accuracy of 0.4455
2019-01-22 09:35:52.622 INFO in 'deeppavlov.core.commands.train'['train'] at line 610: Saving model
2019-01-22 09:35:52.624 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 71: [saving model to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_db/model]
2019-01-22 09:35:52.764 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 708: [saving parameters to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_db/model.json]


...

2019-01-22 10:07:18.60 INFO in 'deeppavlov.core.commands.train'['train'] at line 615: Did not improve on the per_item_dialog_accuracy of 0.4733


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4641}, "time_spent": "0:41:55", "epochs_done": 9, "batches_seen": 2178, "train_examples_seen": 8703, "impatience": 5, "patience_limit": 20}}
{"train": {"epochs_done": 10, "batches_seen": 2420, "train_examples_seen": 9670, "metrics": {"per_item_dialog_accuracy": 0.6284}, "time_spent": "0:45:12", "loss": 0.245588072752657, "learning_rate": 0.002, "momentum": 0.9}}


2019-01-22 10:11:40.23 INFO in 'deeppavlov.core.commands.train'['train'] at line 608: New best per_item_dialog_accuracy of 0.4799
2019-01-22 10:11:40.24 INFO in 'deeppavlov.core.commands.train'['train'] at line 610: Saving model
2019-01-22 10:11:40.25 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 71: [saving model to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_db/model]
2019-01-22 10:11:40.172 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 708: [saving parameters to /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/gobot_dstc2_db/model.json]
2019-01-22 10:11:40.222 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/vocabs/token.dict]
2019-01-22 10:11:40.225 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 48: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-01-22 10:11:40.226 INFO in 'deepp

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4799}, "time_spent": "0:46:17", "epochs_done": 10, "batches_seen": 2420, "train_examples_seen": 9670, "impatience": 0, "patience_limit": 20}}


2019-01-22 10:11:42.220 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 47: [loading model from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/model]
2019-01-22 10:11:42.241 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /Users/maxtran/.deeppavlov/models/slotfill_dstc2/model
2019-01-22 10:11:42.666 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 160: [loading templates from /Users/maxtran/Desktop/CitrusBerry/Personal_Chatbot/dstc2/dstc2-templates.txt]
2019-01-22 10:11:42.669 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 163: 46 templates loaded
2019-01-22 10:11:42.669 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 202: Calculated input size for `GoalOrientedBotNetwork` is 511
2019-01-22 10:11:43.384 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 240: [initializing `GoalOrientedBot` from saved]
2019-01-22 10:11:43.385 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 714: [

In [38]:
model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: i want an expensive french food


2019-01-22 20:57:59.845 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 472: Made api_call with {'pricerange': 'expensive', 'food': 'french'}, got 2 results.


>> Cote serves french food in the expensive price range.
:: phone number?
>> The phone number of cote is 01223 311053.
:: exit
